# Load Data

In [1]:
from LoadData import outcomes_df, projects_df, mega_df

'mega_df' is merged dataframe including: 
- the target variable, 'fully_funded', from 'Outcomes' 
- full 'Projects' dataframe
- indexed by 'projectid'

### Load functions and global variables:

In [2]:
import CleaningFunctions as cf
import ExplorationFunctions as ef
import LoadData as ld
from Variables import *

### Required modules:

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
%matplotlib inline

# Initial cleaning

Remove lat/long and ID variables, change t/f to 1/0, impute the mean for NaN in continuous variables, then bin continuous variables in deciles:

In [4]:
remove_from_projects = GEO_VARIABLES + ID_VARIABLES + IDX
keep_vars = [x for x in projects_df.columns if x not in remove_from_projects]
labeled_df = mega_df[keep_vars + TARGET_VARIABLES]
labeled_df = cf.change_to_1_0(labeled_df)
labeled_df = cf.impute_mean(labeled_df, given_cols=CONTINUOUS_VARIABLES)
for var in CONTINUOUS_VARIABLES:
    labeled_df = cf.discretize(labeled_df, var, 10, want_quantile=True)

/Users/akoko0530/Documents/GitHub/MachineLearning_CAPP30254/Homework_3/CleaningFunctions.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].apply(lambda x: 1 if x=='t' else 0)
/Users/akoko0530/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/akoko0530/Documents/GitHub/MachineLearning_CAPP30254/Homework_3/CleaningFunctions.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

In [5]:
labeled_df.shape

(353151, 30)

In [7]:
labeled_df.head()

,school_city,school_state,school_zip,school_metro,school_district,school_county,school_charter,school_magnet,school_year_round,school_nlns,...,poverty_level,grade_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted,fully_funded
projectid,,,,,,,,,,,,,,,,,,,,,
ffffac55ee02a49d1abc87ba6fc61135,Jonesboro,GA,30236.0,suburban,Clayton Co Public Schools,Clayton,0,0,0,0,...,highest poverty,Grades PreK-2,35.0,2,2,1,0,0,2011-06-11,1
ffff2d9c769c8fb5335e949c615425eb,Pocatello,ID,83204.0,urban,Pocatello-Chubbuck District 25,Bannock,0,0,0,0,...,high poverty,Grades 3-5,30.0,8,8,4,0,0,2013-03-03,1
fffeebf4827d745aa36b17c2d38d1966,Fair Oaks,CA,95628.0,suburban,San Juan Unified School Dist,Sacramento,0,0,0,0,...,highest poverty,Grades 6-8,30.0,9,9,1,0,0,2012-12-01,1
fffeaae9482c9b72cab5bbd3f061d362,Yukon,OK,73099.0,suburban,Yukon School District I-27,Canadian,0,0,0,0,...,high poverty,Grades PreK-2,35.0,6,6,8,0,0,2011-07-26,0
fffe45b28ea6f2889de1e3f797fb31a3,Westminster,CO,80030.0,suburban,Adams 12 Five Star Schools,Adams,0,0,0,0,...,highest poverty,Grades 3-5,35.0,3,3,10,0,0,2012-01-14,1


In [6]:
cf.has_na(labeled_df)

[('school_metro', 43021),
 ('school_district', 429),
 ('primary_focus_subject', 35),
 ('primary_focus_area', 35),
 ('secondary_focus_subject', 116086),
 ('secondary_focus_area', 116086),
 ('resource_type', 42),
 ('grade_level', 6)]

- Lots of missing observations from school_metro, secondary_focus_subject, secondary_focus_area

In [ ]:
labeled_df = cf.dummytize(labeled_df, dum_variables_of_interest)

In [5]:
funded_df = labeled_df[labeled_df['fully_funded']==1]
nonfunded_df = labeled_df[labeled_df['fully_funded']==0]

# Exploration and more cleaning

In [6]:
funded_df.describe()

,school_zip,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_teach_for_america,teacher_ny_teaching_fellow,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,fully_funded
count,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247698.000000,247632.000000,247698.000000,247698.000000,247698.0
mean,56383.374747,0.099004,0.088099,0.048729,0.013036,0.007307,0.006702,0.056989,0.011474,32.207587,482.121246,567.201488,92.588632,0.352021,0.050856,1.0
std,31497.564702,0.298667,0.283440,0.215301,0.113429,0.085170,0.081589,0.231822,0.106499,2.482845,767.469691,902.905514,156.042742,0.477602,0.219705,0.0
min,410.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,37.030000,43.560000,1.000000,0.000000,0.000000,1.0
25%,28752.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,257.920000,303.440000,23.000000,0.000000,0.000000,1.0
50%,60620.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,395.530000,465.330000,30.000000,0.000000,0.000000,1.0
75%,90022.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.000000,541.747500,637.347500,100.000000,1.000000,0.000000,1.0
max,99926.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.000000,139725.410000,164382.840000,12143.000000,1.000000,1.000000,1.0


In [7]:
nonfunded_df.describe()

,school_zip,school_charter,school_magnet,school_year_round,school_nlns,school_kipp,school_charter_ready_promise,teacher_teach_for_america,teacher_ny_teaching_fellow,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,fully_funded
count,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105453.000000,105418.000000,105453.000000,105453.000000,105453.0
mean,56110.067983,0.083734,0.077911,0.049899,0.008857,0.005064,0.004741,0.030525,0.007245,32.250197,647.058084,761.244744,97.942211,0.234825,0.028828,0.0
std,29949.240035,0.276990,0.268034,0.217737,0.093695,0.070981,0.068695,0.172029,0.084809,2.487500,991.632938,1166.627001,159.122005,0.423891,0.167324,0.0
min,410.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,45.150000,53.120000,1.000000,0.000000,0.000000,0.0
25%,29801.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,369.540000,434.750000,22.000000,0.000000,0.000000,0.0
50%,60053.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,481.120000,566.020000,31.000000,0.000000,0.000000,0.0
75%,89101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.000000,732.950000,862.290000,100.000000,0.000000,0.000000,0.0
max,99901.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,35.000000,63530.690000,74741.990000,999.000000,1.000000,1.000000,0.0


Variables with notable difference in mean, between funded and non-funded:
- total_price_excluding_optional_support
- total_price_including_optional_support

In [ ]:
# funded_df = f.drop_missing(funded_df)

### Top 5 results by count of each attribute, given that project was fully funded:

In [13]:
cols_of_interst = list(funded_df.columns)[:-2]
for col in cols_of_interst:
    print(col)
    print(ef.find_most_funded(funded_df, col, 'fully_funded', 5), '\n')

school_city
fully_funded  school_city
1             Los Angeles    10797
              Chicago         7633
              Brooklyn        5749
              Bronx           4666
              Houston         4500
Name: school_city, dtype: int64 

school_state
fully_funded  school_state
1             CA              55816
              NY              20677
              TX              15310
              FL              14411
              IL              11639
Name: school_state, dtype: int64 

school_zip
fully_funded  school_zip
1             90011.0       832
              73160.0       776
              64801.0       679
              91340.0       664
              10451.0       640
Name: school_zip, dtype: int64 

school_metro
fully_funded  school_metro
1             urban           130755
              suburban         62593
              rural            25781
Name: school_metro, dtype: int64 

school_district
fully_funded  school_district              
1             Los Angel

In [22]:
labeled_df.columns

Index(['school_city', 'school_state', 'school_zip', 'school_metro',
       'school_district', 'school_county', 'school_charter', 'school_magnet',
       'school_year_round', 'school_nlns', 'school_kipp',
       'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted', 'fully_funded'],
      dtype='object')

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

# Generate a large random dataset
rs = np.random.RandomState(33)
d = pd.DataFrame(data=rs.normal(size=(100, 26)),
                 columns=list(ascii_letters[26:]))

# Compute the correlation matrix
corr = d.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [20]:
def correlation_matrix(df):
    sns.set(style="white")
    # Generate a large random dataset
    d = pd.DataFrame(data=rs.normal(size=(100, 26)),
                     xticklabels=corr.columns.values,
                     yticklabels=corr.columns.values)
    # Compute the correlation matrix
    corr = d.corr()
    # Generate a mask for the upper triangle
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True
    # Set up the matplotlib figure
    f, ax = plt.subplots(figsize=(11, 9))
    # Generate a custom diverging colormap
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    # Draw the heatmap with the mask and correct aspect ratio
    sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [21]:
correlation_matrix(funded_df)

UnboundLocalError: local variable 'corr' referenced before assignment